# **Welcome to our project report! ✨🧪**
## 🚀 *Overview*
This notebook presents an analysis of our pKa prediction package. 

## 🤯 *Acquiring Dataset*
In a first step, we will acquire the pKa dataset from cbio3lab's repository at the following source: https://github.com/cbio3lab/pKa/blob/main/Data/test_acids_bases_descfinal_nozwitterions.csv. The latter was extracted from the Harvard dataverse: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/6A67L9